# **Power System Simulation Final Presentation FUSE**

## Introduction
Welcome to the final presentation on Power System Simulation. In this presentation, we will cover the following packages:

* `graph_processing.py`
* `power_grid_modelling.py`
* `grid_analytic.py`

We have tested all the packages using `pytest` to ensure there are no errors and that the output is as expected.


In [1]:
print("Hello world!")

Hello world!


hello